In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle

In [3]:
# load the data
team_df= pd.read_parquet('parquet_data/team_df_missing_handled.parquet')
player_df= pd.read_parquet('parquet_data/player_df_missing_handled.parquet')

In [4]:
# Eğer tarih formatı 'dd/mm/yyyy' ise
player_df['Date'] = pd.to_datetime(player_df['Date'], format='%m/%d/%Y')
team_df['Date'] = pd.to_datetime(team_df['Date'], format='%m/%d/%Y')

In [5]:
player_df['Assists'] = np.log1p(player_df['Assists'])  # log(assists + 1) dönüşümü
team_df['Assists'] = np.log1p(team_df['Assists'])  # log(assists + 1) dönüşümü

In [6]:
# Team_df'teki duplicaleri droplayalım
grouped = team_df.groupby(['name', 'Date', 'Opponent'])
# Her grup için eleman sayısı
group_sizes = grouped.size()
two_element_groups = group_sizes[group_sizes == 2]

# 2 elemanlı grupların içerdiği indeksleri bulma
# grouped.indices bu grupların indekslerini döndürür
two_element_indices = [index for group in two_element_groups.index for index in grouped.get_group(group).index]

# Orijinal veri setinden bu indeksleri drop etme
team_df_dropped = team_df.drop(two_element_indices)

In [7]:
# find the matches that the player is playing from tema_df
merged_data = pd.merge(player_df, team_df_dropped, 
                       left_on=['team_name', 'Date', 'Opponent'], 
                       right_on=['name', 'Date', 'Opponent'], 
                       how='left')

In [8]:
player_df

,Date,Opponent,Result,S,Kills,Errors,Total Attacks,Hit Pct,Assists,SErr,Digs,Block Assists,PTS,name,team_name
0,2016-09-17,@ Yale,L 1 - 3,4.0,14.0,3.0,37.0,0.297,1.386294,1.0,5.0,3.0,16.5,"Carlson, Morgan",A&M-Corpus Christi (Southland)
1,2016-10-15,Lamar University,W 3 - 0,3.0,8.0,3.0,20.0,0.250,1.386294,1.0,3.0,4.0,10.0,"Carlson, Morgan",A&M-Corpus Christi (Southland)
2,2016-10-20,@ New Orleans,W 3 - 1,4.0,9.0,3.0,21.0,0.286,0.693147,2.0,5.0,3.0,11.5,"Carlson, Morgan",A&M-Corpus Christi (Southland)
3,2016-11-03,McNeese,W 3 - 0,3.0,9.0,1.0,17.0,0.471,0.693147,1.0,6.0,2.0,10.0,"Carlson, Morgan",A&M-Corpus Christi (Southland)
4,2016-11-12,@ Houston Baptist,W 3 - 1,4.0,7.0,6.0,21.0,0.048,0.693147,2.0,8.0,4.0,9.0,"Carlson, Morgan",A&M-Corpus Christi (Southland)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30796,2019-11-08,@ Green Bay,L 0 - 3,3.0,9.0,8.0,32.0,0.031,0.693147,2.0,12.0,3.0,13.5,"Pe'a, Brooklen",Youngstown St. (Horizon)
30797,2019-10-05,Milwaukee,L 0 - 3,3.0,10.0,3.0,38.0,0.184,0.693147,3.0,12.0,1.0,10.5,"Thompson, Margaux",Youngstown St. (Horizon)
30798,2019-10-09,@ IUPUI,W 3 - 2,5.0,17.0,6.0,37.0,0.297,0.693147,5.0,21.0,1.0,20.5,"Thompson, Margaux",Youngstown St. (Horizon)
30799,2019-10-11,Green Bay,L 0 - 3,3.0,3.0,7.0,26.0,-0.154,0.693147,2.0,12.0,1.0,3.5,"Thompson, Margaux",Youngstown St. (Horizon)


In [9]:
merged_data

,Date,Opponent,Result_x,S_x,Kills_x,Errors_x,Total Attacks_x,Hit Pct_x,Assists_x,SErr_x,...,Total Attacks_y,Hit Pct_y,Assists_y,Aces,SErr_y,Digs_y,RErr,Block Assists_y,PTS_y,name_y
0,2016-09-17,@ Yale,L 1 - 3,4.0,14.0,3.0,37.0,0.297,1.386294,1.0,...,143.0,0.140,3.663562,3.0,4.0,56.0,6.0,18.0,57.0,A&M-Corpus Christi (Southland)
1,2016-10-15,Lamar University,W 3 - 0,3.0,8.0,3.0,20.0,0.250,1.386294,1.0,...,111.0,0.270,3.912023,4.0,5.0,47.0,3.0,12.0,60.0,A&M-Corpus Christi (Southland)
2,2016-10-20,@ New Orleans,W 3 - 1,4.0,9.0,3.0,21.0,0.286,0.693147,2.0,...,138.0,0.312,4.077537,3.0,7.0,67.0,5.0,20.0,77.0,A&M-Corpus Christi (Southland)
3,2016-11-03,McNeese,W 3 - 0,3.0,9.0,1.0,17.0,0.471,0.693147,1.0,...,102.0,0.324,3.713572,7.0,5.0,51.0,1.0,8.0,55.0,A&M-Corpus Christi (Southland)
4,2016-11-12,@ Houston Baptist,W 3 - 1,4.0,7.0,6.0,21.0,0.048,0.693147,2.0,...,137.0,0.248,3.951244,3.0,15.0,67.0,2.0,16.0,65.0,A&M-Corpus Christi (Southland)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30796,2019-11-08,@ Green Bay,L 0 - 3,3.0,9.0,8.0,32.0,0.031,0.693147,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30797,2019-10-05,Milwaukee,L 0 - 3,3.0,10.0,3.0,38.0,0.184,0.693147,3.0,...,123.0,0.073,3.218876,1.0,4.0,58.0,3.0,12.0,31.0,Youngstown St. (Horizon)
30798,2019-10-09,@ IUPUI,W 3 - 2,5.0,17.0,6.0,37.0,0.297,0.693147,5.0,...,163.0,0.166,3.951244,8.0,13.0,81.0,9.0,6.0,68.0,Youngstown St. (Horizon)
30799,2019-10-11,Green Bay,L 0 - 3,3.0,3.0,7.0,26.0,-0.154,0.693147,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Let's calculate the performance rates
merged_data['Kill_ratio'] = 100*(merged_data['Kills_x'] / merged_data['Kills_y'])
merged_data['Errors_ratio'] =100*( merged_data['Errors_x'] / merged_data['Errors_y'])
#merged_data['Hit_pct_ratio'] = merged_data['Hit Pct_x'] / merged_data['Hit Pct_y'] # hit_pct_ is already a rate
merged_data['Assists_ratio'] =100*( merged_data['Assists_x'] / merged_data['Assists_y'])
merged_data['Total Attacks_ratio'] =100*( merged_data['Total Attacks_x'] / merged_data['Total Attacks_y'])
merged_data['PTS_ratio'] = 100*(merged_data['PTS_x'] / merged_data['PTS_y'])
merged_data['SErr_ratio'] = 100*(merged_data['SErr_x'] / merged_data['SErr_y'])
merged_data['Digs_ratio'] = 100*(merged_data['Digs_x'] / merged_data['Digs_y'])
merged_data['Block Assists_ratio'] = 100*(merged_data['Block Assists_x'] / merged_data['Block Assists_y'])
#merged_data['S_ratio'] = merged_data['S_x'] / merged_data['S_y'] # don't need to calculate rate of set column

In [11]:
# Create the new data set and select the necessary columns (select the columns that in player data set)
player_performance_ratios = merged_data[['Date','Opponent','Result_x','S_x', 'Kill_ratio', 'Errors_ratio', 'Total Attacks_ratio','Hit Pct_x', 'Assists_ratio', 
                                         'SErr_ratio', 'Digs_ratio', 'Block Assists_ratio', 
                                         'PTS_ratio', 'name_x','name_y']]

In [12]:
rows_to_drop = player_performance_ratios[player_performance_ratios.isnull().any(axis=1)].index
# Drop the corresponding rows in the player_df dataset
player_df_cleaned = player_df.drop(rows_to_drop)

player_performance_ratios=player_performance_ratios.dropna() # drop null rows (there is matches which are in player dataset but not in team dataset)
player_performance_ratios = player_performance_ratios.rename(columns={'name_x': 'name', 'name_y': 'team_name'})
player_performance_ratios

,Date,Opponent,Result_x,S_x,Kill_ratio,Errors_ratio,Total Attacks_ratio,Hit Pct_x,Assists_ratio,SErr_ratio,Digs_ratio,Block Assists_ratio,PTS_ratio,name,team_name
0,2016-09-17,@ Yale,L 1 - 3,4.0,31.818182,12.500000,25.874126,0.297,37.840072,25.000000,8.928571,16.666667,28.947368,"Carlson, Morgan",A&M-Corpus Christi (Southland)
1,2016-10-15,Lamar University,W 3 - 0,3.0,16.326531,15.789474,18.018018,0.250,35.436764,20.000000,6.382979,33.333333,16.666667,"Carlson, Morgan",A&M-Corpus Christi (Southland)
2,2016-10-20,@ New Orleans,W 3 - 1,4.0,14.516129,15.789474,15.217391,0.286,16.999162,28.571429,7.462687,15.000000,14.935065,"Carlson, Morgan",A&M-Corpus Christi (Southland)
3,2016-11-03,McNeese,W 3 - 0,3.0,20.454545,9.090909,16.666667,0.471,18.665241,20.000000,11.764706,25.000000,18.181818,"Carlson, Morgan",A&M-Corpus Christi (Southland)
4,2016-11-12,@ Houston Baptist,W 3 - 1,4.0,13.207547,31.578947,15.328467,0.048,17.542506,13.333333,11.940299,25.000000,13.846154,"Carlson, Morgan",A&M-Corpus Christi (Southland)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30794,2019-09-10,Kent St.,L 2 - 3,5.0,37.500000,9.090909,32.620321,0.311,17.629143,15.384615,12.658228,10.000000,30.666667,"Pe'a, Brooklen",Youngstown St. (Horizon)
30795,2019-09-20,Eastern Ill.,W 3 - 1,4.0,16.666667,17.857143,22.388060,0.100,18.104260,18.181818,14.754098,12.500000,16.666667,"Pe'a, Brooklen",Youngstown St. (Horizon)
30797,2019-10-05,Milwaukee,L 0 - 3,3.0,41.666667,20.000000,30.894309,0.184,21.533828,75.000000,20.689655,8.333333,33.870968,"Thompson, Margaux",Youngstown St. (Horizon)
30798,2019-10-09,@ IUPUI,W 3 - 2,5.0,31.481481,22.222222,22.699387,0.297,17.542506,38.461538,25.925926,16.666667,30.147059,"Thompson, Margaux",Youngstown St. (Horizon)


In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import numpy as np

In [14]:
target_columns = ['Kill_ratio','Errors_ratio','Total Attacks_ratio','Assists_ratio','Digs_ratio','Block Assists_ratio','PTS_ratio']
# Verilerinizi hazırlayalım (Önceden temizlenmiş ve işlenmiş X ve y verisi)
player_df_columns =['Kills', 'Errors', 'Total Attacks', 'Assists','Digs','Block Assists', 'PTS']

In [15]:
X_full = player_df_cleaned[player_df_columns]

for idx, column in enumerate(target_columns):
    y_column = player_performance_ratios[column]
    
    # X (özellikler) ve y (hedef) verilerini ayıralım
    X = X_full.drop(columns=[X_full.columns[idx]])

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)  # X_scaled = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_column, test_size=0.2, random_state=42)
    
    # Random Forest modelini oluşturun ve eğitin
    model_random_original = RandomForestRegressor(n_estimators=100, random_state=42)
    model_random_original.fit(X_train, y_train)

    # Dosya yolu
    file_path = f'pkl_files_regression/randomForest_model_{X_full.columns[idx]}.pkl'

    # Klasör yoksa oluştur
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # Modeli kaydet
    with open(file_path, 'wb') as f:
        pickle.dump(model_random_original, f)
    
    y_pred = model_random_original.predict(X_test)
    
    # Model değerlendirme metriklerini hesaplayalım
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Sonuçları yazdıralım
    print(f"Değerlendirme sonuçları ({column} için):")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R-squared (R²): {r2:.4f}")
    print("-" * 50)

Değerlendirme sonuçları (Kill_ratio için):
Mean Squared Error (MSE): 17.8160
Root Mean Squared Error (RMSE): 4.2209
Mean Absolute Error (MAE): 2.9359
R-squared (R²): 0.8448
--------------------------------------------------
Değerlendirme sonuçları (Errors_ratio için):
Mean Squared Error (MSE): 74.3930
Root Mean Squared Error (RMSE): 8.6251
Mean Absolute Error (MAE): 6.3406
R-squared (R²): 0.5462
--------------------------------------------------
Değerlendirme sonuçları (Total Attacks_ratio için):
Mean Squared Error (MSE): 17.6770
Root Mean Squared Error (RMSE): 4.2044
Mean Absolute Error (MAE): 3.0925
R-squared (R²): 0.8323
--------------------------------------------------
Değerlendirme sonuçları (Assists_ratio için):
Mean Squared Error (MSE): 336.7775
Root Mean Squared Error (RMSE): 18.3515
Mean Absolute Error (MAE): 11.3464
R-squared (R²): 0.7223
--------------------------------------------------
Değerlendirme sonuçları (Digs_ratio için):
Mean Squared Error (MSE): 47.0734
Root Mean 